In [1]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

/Users/sajanakadangara/code_base/research_project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df = pd.read_csv("data/raw_data.csv")
df.head()

,payment_type,profit_per_order,sales_per_customer,category_id,category_name,customer_city,customer_country,customer_id,customer_segment,customer_state,...,order_region,order_state,order_status,product_card_id,product_category_id,product_name,product_price,shipping_date,shipping_mode,label
0,DEBIT,34.448338,92.49099,9.0,Cardio Equipment,Caguas,Puerto Rico,12097.6830,Consumer,PR,...,Western Europe,Vienna,COMPLETE,191.0,9.0,Nike Men's Free 5.0+ Running Shoe,99.99,2015-08-13 00:00:00+01:00,Standard Class,-1
1,TRANSFER,91.193540,181.99008,48.0,Water Sports,Albuquerque,EE. UU.,5108.1045,Consumer,CA,...,South America,Buenos Aires,PENDING,1073.0,48.0,Pelican Sunstream 100 Kayak,199.99,2017-04-09 00:00:00+01:00,Standard Class,-1
2,DEBIT,8.313806,89.96643,46.0,Indoor/Outdoor Games,Amarillo,Puerto Rico,4293.4478,Consumer,PR,...,Western Europe,Nord-Pas-de-Calais-Picardy,COMPLETE,1014.0,46.0,O'Brien Men's Neoprene Life Vest,49.98,2015-03-18 00:00:00+00:00,Second Class,1
3,TRANSFER,-89.463196,99.15065,17.0,Cleats,Caguas,Puerto Rico,546.5306,Consumer,PR,...,Central America,Santa Ana,PROCESSING,365.0,17.0,Perfect Fitness Perfect Rip Deck,59.99,2017-03-18 00:00:00+00:00,Second Class,0
4,DEBIT,44.722590,170.97824,48.0,Water Sports,Peabody,EE. UU.,1546.3980,Consumer,CA,...,Central America,Illinois,COMPLETE,1073.0,48.0,Pelican Sunstream 100 Kayak,199.99,2015-03-30 00:00:00+01:00,Standard Class,1


In [3]:
# Display the features available
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15549 entries, 0 to 15548
Data columns (total 41 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type              15549 non-null  object 
 1   profit_per_order          15549 non-null  float64
 2   sales_per_customer        15549 non-null  float64
 3   category_id               15549 non-null  float64
 4   category_name             15549 non-null  object 
 5   customer_city             15549 non-null  object 
 6   customer_country          15549 non-null  object 
 7   customer_id               15549 non-null  float64
 8   customer_segment          15549 non-null  object 
 9   customer_state            15549 non-null  object 
 10  customer_zipcode          15549 non-null  float64
 11  department_id             15549 non-null  float64
 12  department_name           15549 non-null  object 
 13  latitude                  15549 non-null  float64
 14  longit

In [ ]:
# remove unwanted columns
'''payment_type,	profit_per_order,sales_per_customer,	category_id,category_name, customer_id,customer_segment,customer_zipcode	department_id, department_name,market,	order_customer_id,order_date,order_id, order_item_cardprod_id,order_item_discount,	order_item_discount_rate,	order_item_id,	order_item_product_price,	order_item_profit_ratio,	order_item_quantity,	sales,	order_item_total_amount,	order_profit_per_order,	order_region,	order_status,	product_card_id,	product_category_id,	product_name,	product_price,	shipping_date,	label, order_state'''

In [4]:
df.drop(columns=["payment_type","profit_per_order","sales_per_customer","category_id","category_name","customer_id","customer_segment","customer_zipcode","department_id","department_name","market","order_customer_id","order_date",
  "order_id","order_item_cardprod_id","order_item_discount","order_item_discount_rate","order_item_id","order_item_product_price",
"order_item_profit_ratio","order_item_quantity","sales","order_item_total_amount","order_profit_per_order","order_region","order_status","product_card_id","product_category_id","product_name","product_price","shipping_date","label","order_state",
                 "customer_state"
 ], inplace=True)

In [5]:
df.head()

,customer_city,customer_country,customer_state,latitude,longitude,order_city,order_country,shipping_mode
0,Caguas,Puerto Rico,PR,18.359064,-66.370575,Viena,Austria,Standard Class
1,Albuquerque,EE. UU.,CA,37.636528,-121.119630,Buenos Aires,Argentina,Standard Class
2,Amarillo,Puerto Rico,PR,18.294100,-66.037056,Burnie,France,Second Class
3,Caguas,Puerto Rico,PR,18.202435,-66.370510,Santa Ana,El Salvador,Second Class
4,Peabody,EE. UU.,CA,38.719500,-122.319720,Blumenau,Mexico,Standard Class


In [ ]:
# Drop rows where column above column is NaN
df_no_duplicates = df.drop_duplicates()
df_cleaned  = df_no_duplicates.dropna()

In [ ]:
# to calculate the total distance, we need to get the destination city latitude and longitude

# function for calculating latitude and longitude
def get_lat_lon(city_name):
    # Use a clear and unique user agent string
    geolocator = Nominatim(
        user_agent="writetosajana@gmail.com",
        timeout=100
    )
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    else:
        return None

In [ ]:
df_cleaned[['dest_lat','dest_lon']] = df_cleaned['order_city'].apply(get_lat_lon)

In [ ]:
# let's rename columns to proper names
df_cleaned.rename(columns={"customer_city":"source",
                           "customer_country":"source_country",
                           "latitude":"source_latitude",
                           "longitude":"source_longitude",
                           "order_city":"destination",
                            "order_country":"destination_country",
                },inplace=True)

In [ ]:
# drop duplicate values

df_no_duplicates.to_excel("data/final_data.xlsx", index=False)
